# Initial Exploratory Data Analysis

## Load necessary packages

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

## Helper functions

In [3]:
def expand_date(dataframe):
    """expands the date to add columns with year, month, day, weekday
    
    Args:
        dataframe: a pandas dataframe with column 'date', among others
    
    Returns:
        a copy of the dataframe with out the date column, and with the
            new year, month, day, weekday columns
    """
    df_copy = dataframe.copy()
    X = pd.to_datetime(df_copy['date'], format="%Y-%m-%d")
    df_copy['year'] = [x.year for x in X]
    df_copy['month'] = [x.month for x in X]
    df_copy['day'] = [x.day for x in X]
    df_copy['weekday'] = [x.weekday() for x in X]
    df_copy = df_copy.drop(columns=['date'])
    print(df_copy.head())
    return df_copy

In [4]:
def plot_w_best_fit_line(x, y):
    """plots two-dimensional data with a dashed best fit line.
    
    Args:
        x: x-values
        y: y-values
        
    Returns:
        none
    """
    coef = np.polyfit(x,y,1)
    poly1d_fn = np.poly1d(coef) 
    # poly1d_fn is now a function which takes in x and returns an estimate for y

    plt.plot(x,y, 'yo', x, poly1d_fn(x), '--k')

## Load data from SQL database

In [5]:
df = pd.read_sql_table('manhattan_loc_d_ar_wea', 'postgresql:///walk')

## Data Cleaning

In [6]:
# fill n_arrests with 0 in place of NaN
df['n_arrests'] = df['n_arrests'].fillna(value=0).astype(int)

In [7]:
# expand date features
df_expanded = expand_date(df)

   latitude  longitude  n_arrests  ap_t_high100  ap_t_low100  cloud  humidity  \
0    40.683    -74.023          0          3974         3626     95        82   
1    40.684    -74.012          0          3974         3626     95        82   
2    40.684    -74.024          0          3974         3626     95        82   
3    40.685    -74.022          0          3974         3626     95        82   
4    40.685    -74.023          0          3974         3626     95        82   

   precip_inten_max10000  precip_proba100  sunriseTime  sunsetTime  \
0                     17               13   1136118060  1136151594   
1                     17               13   1136118060  1136151594   
2                     17               13   1136118060  1136151594   
3                     17               13   1136118060  1136151594   
4                     17               13   1136118060  1136151594   

   wind_gust100  precip_accum100  year  month  day  weekday  
0           454              N

In [10]:
# ozone levels were not recorded for the earlier years of the data set
# they do not seem to be correlated with the arrest count
# decided to drop ozone readings.
# df_expanded = df_expanded.drop(columns=['ozone10'])

# the remaining features all have NaN in place of zero. Filling NaNs with 0s
df_expanded.fillna(0)

,latitude,longitude,n_arrests,ap_t_high100,ap_t_low100,cloud,humidity,precip_inten_max10000,precip_proba100,sunriseTime,sunsetTime,wind_gust100,precip_accum100,year,month,day,weekday
0,40.683,-74.023,0,3974,3626,95,82,17,13,1136118060,1136151594,454,0.0,2006,1,1,6
1,40.684,-74.012,0,3974,3626,95,82,17,13,1136118060,1136151594,454,0.0,2006,1,1,6
2,40.684,-74.024,0,3974,3626,95,82,17,13,1136118060,1136151594,454,0.0,2006,1,1,6
3,40.685,-74.022,0,3974,3626,95,82,17,13,1136118060,1136151594,454,0.0,2006,1,1,6
4,40.685,-74.023,0,3974,3626,95,82,17,13,1136118060,1136151594,454,0.0,2006,1,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111843,40.878,-73.909,0,8357,6475,37,58,39,29,1561886932,1561941130,1425,0.0,2019,6,30,6
31111844,40.878,-73.910,0,8357,6475,37,58,39,29,1561886932,1561941130,1425,0.0,2019,6,30,6
31111845,40.878,-73.911,0,8357,6475,37,58,39,29,1561886932,1561941130,1425,0.0,2019,6,30,6
31111846,40.878,-73.912,0,8357,6475,37,58,39,29,1561886932,1561941130,1425,0.0,2019,6,30,6


In [11]:
from statsmodels.formula.api import ols

## Looking at single-feature linear regression for each feature

In [13]:
for col in df_expanded.columns:
    outcome = 'n_arrests'
    predictor = col
    formula = outcome + '~' + predictor
    model = ols(formula=formula, data=df_expanded).fit()
    print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              n_arrests   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     217.3
Date:                Tue, 26 Nov 2019   Prob (F-statistic):           3.53e-49
Time:                        13:59:26   Log-Likelihood:            -9.0778e+06
No. Observations:            31111848   AIC:                         1.816e+07
Df Residuals:                31111846   BIC:                         1.816e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8245      0.053     15.537      0.0

                            OLS Regression Results                            
Dep. Variable:              n_arrests   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     212.3
Date:                Tue, 26 Nov 2019   Prob (F-statistic):           4.25e-48
Time:                        14:02:34   Log-Likelihood:            -9.0778e+06
No. Observations:            31111848   AIC:                         1.816e+07
Df Residuals:                31111846   BIC:                         1.816e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0437      0.000    387.241      0.0

                            OLS Regression Results                            
Dep. Variable:              n_arrests   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                 1.059e+04
Date:                Tue, 26 Nov 2019   Prob (F-statistic):               0.00
Time:                        14:05:34   Log-Likelihood:            -9.0726e+06
No. Observations:            31111848   AIC:                         1.815e+07
Df Residuals:                31111846   BIC:                         1.815e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1078      0.001    168.507      0.0

                            OLS Regression Results                            
Dep. Variable:              n_arrests   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     471.9
Date:                Tue, 26 Nov 2019   Prob (F-statistic):          1.23e-104
Time:                        14:08:57   Log-Likelihood:            -9.0777e+06
No. Observations:            31111848   AIC:                         1.816e+07
Df Residuals:                31111846   BIC:                         1.816e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0445      0.000    374.210      0.0